In [1]:
import pandas as pd
import numpy as np

In [2]:
import numpy as np
import scipy as sp
from scipy import stats

def jsd(p, q, base=np.e):
    '''
        Implementation of pairwise `jsd` based on  
        https://en.wikipedia.org/wiki/Jensen%E2%80%93Shannon_divergence
    '''
    ## convert to np.array
    p, q = np.asarray(p), np.asarray(q)
    ## normalize p, q to probabilities
    p, q = p/p.sum(), q/q.sum()
    m = 1./2*(p + q)
    return stats.entropy(p,m, base=base)/2. +  stats.entropy(q, m, base=base)/2.

In [3]:
def populateJSD(inputlist,df):
    for item in inputlist:
        col_name = str(item)
        df[col_name] = (list(map(lambda x:jsd(df.loc[df[df.name == item].index[0],
                                                               'topicDistribution'],
                                                 x) ,df.topicDistribution)))
    return df

In [4]:
def populateJSDWithID(inputlist,df):
    for item in inputlist:
        col_name =df1.loc[item,'name'] + " " + df1.loc[item,'address'][:15]
        df[col_name] = (list(map(lambda x:jsd(df.loc[item,'topicDistribution'],x) ,df.topicDistribution)))
    return df

In [5]:
rest_df = pd.read_csv("entity.csv",header=0)
print("shape ",rest_df.shape)
rest_df.head()

shape  (9808, 3)


,id,name,address
0,service_b82aaa6ef08a4555980c8766e74a3580,Afters,"Shop 192, Al Seef Mall, Al Seef Street, Umm Hu..."
1,service_9fb456d7a66d4e0893ef104115f2d40f,Jelly Belly Ice Cream,"Building 10, Phase 3, Al Seef Mall, Al Seef St..."
2,service_48cbfc508de4471097ff0d3c9539f0d2,Kcal,"Canal Residence, Promenade Retail Area, Dubai ..."
3,service_05f22cc215644293b6e471fa75b2e3ff,Kohantei,"Dubai Opera, Plaza Unit 6, Sheikh Mohammed Bin..."
4,service_5e87c0a1ae104ff5a8157f59747d5ef0,Sun Tour Restaurant,"Ground Level, A 03 Building, China Cluster, In..."


# Only Zomato Dubai with 180 topics

In [30]:
import ast
with open('reviews-zomato-dubai-topic-composition-spark.txt') as f:
    data = ast.literal_eval(f.read())
df1 = pd.DataFrame(data)
print(df1.shape)
df1.head()

(8847, 2)


,id,topicDistribution
0,service_00026cb6595b4e93bb530c64bb4df92f,"[0.0005571014085451798, 0.003470435993264386, ..."
1,service_000726bc5df5449aa1b4555d54b754ff,"[0.00029594279024264676, 0.004474207121988655,..."
2,service_000b662279ce4ade9e3fd48f912c7aee,"[8.036530215281802e-05, 0.01300210717370562, 8..."
3,service_00199d7edaa14e9285d450f516538ba5,"[1.1849668326396843e-05, 4.465561080808407e-05..."
4,service_001bd5d853a8463b987ac5942a70436a,"[0.0003507513219250869, 0.00249677231122796, 3..."


In [31]:
df1 = df1.merge(rest_df,on='id',how='inner')
print("Shape ",df1.shape)
df1.set_index('id',inplace=True)
df1.head()

Shape  (8334, 4)


,topicDistribution,name,address
id,,,
service_00026cb6595b4e93bb530c64bb4df92f,"[0.0005571014085451798, 0.003470435993264386, ...",Lywa Oriental Foods,"Food Court, Al Barsha Mall, Barsha 2, Al Barsh..."
service_000726bc5df5449aa1b4555d54b754ff,"[0.00029594279024264676, 0.004474207121988655,...",Sprouts Restaurant & Cafe,"Shop 2, Eros House Badri Building, Near Abidos..."
service_000b662279ce4ade9e3fd48f912c7aee,"[8.036530215281802e-05, 0.01300210717370562, 8...",Costa Coffee,"Premier Inn, Dubai Investment Park, Dubai"
service_00199d7edaa14e9285d450f516538ba5,"[1.1849668326396843e-05, 4.465561080808407e-05...",Hayz,"Ground Floor, JLT Cluster R, Jumeirah Lake Tow..."
service_001bd5d853a8463b987ac5942a70436a,"[0.0003507513219250869, 0.00249677231122796, 3...",Qasr Al Jabal,"EPPCO Station, Damascus Street, Al Qusais Indu..."


In [25]:
df1[df1.name == 'The Cheesecake Factory']

,topicDistribution,name,address
id,,,
service_a4a2876168514be8aef0b8513ac11d9e,"[2.1659650203489473e-06, 1.3174702148888526e-0...",The Cheesecake Factory,"The Beach, Jumeirah Beach Residence (JBR), Dubai"
service_db4c591268854b8e91ac2c1ca857b5f2,"[3.8106295946837056e-07, 0.01029361454483183, ...",The Cheesecake Factory,"Ground Level, Near Aquarium, The Dubai Mall, D..."
service_fa5366f9aab94da7afb01b9fa590c187,"[5.910337042640526e-07, 0.009440938322053692, ...",The Cheesecake Factory,"Ground Level, Beside Kempinski Hotel, Mall of ..."
service_d21a0be7b8004cf2826d8c47cd8351c3,"[2.4012034116012733e-06, 0.005173895412442235,...",The Cheesecake Factory,"Level 1, Dubai Festival City Mall, Dubai Festi..."


In [26]:
df1[df1.name.str.contains('Taqado Mexican Kitchen',case=False)]

,topicDistribution,name,address
id,,,
service_eaaa434f61714d3db0aee20e5804eb3f,"[4.910792593708519e-05, 0.0004805242635086005,...",Taqado Mexican Kitchen,Delivery Only
service_91c88624be944208b0dbc3ee585cebbd,"[5.051774597807891e-06, 0.017400879998269343, ...",Taqado Mexican Kitchen,"I-Rise Tower, First Al Khail Road, Barsha Heig..."
service_c50af7a1ef5c4bdd8b1401c1597e77d7,"[3.614357831074346e-05, 0.012498109926155732, ...",Taqado Mexican Kitchen,"Concourse D, Terminal 1, Dubai International A..."
service_f13de27eea3e402d9b7cf0d96ab8dc72,"[1.2676797426870601e-05, 0.009420514761684765,...",Taqado Mexican Kitchen,"Ground Floor, CNBC Building 7, Al Falak Street..."
service_812f15e9e0284fc4b73dbd9bd1c7613a,"[9.089727353454668e-06, 0.008683473200031983, ...",Taqado Mexican Kitchen,"Ground Floor, Marble Walk, Building 5, DIFC, D..."
service_27bef048cd914519ace7a3ee56b107b6,"[1.578749091896677e-05, 0.010124302802855123, ...",Taqado Mexican Kitchen,"Ground Level, Bay Avenue, Executive Tower, Bus..."
service_0f278058e4864615bc26a04d047e8935,"[2.9630776536964924e-05, 0.030453844083504447,...",Taqado Mexican Kitchen,"China Court, Ibn Battuta Mall, Garden Cross Ro..."
service_b774b4ad9a7e486c9f3e25a2cb825246,"[0.00018211399104447782, 0.0012963519790359695...",Taqado Mexican Kitchen,Delivery Only
service_b81f8665be684648879a70001668b78b,"[2.554503960022417e-06, 0.06743322313839745, 2...",Taqado Mexican Kitchen,"East Food Court, Ground Level, Mall of the Emi..."


In [32]:
inputlist = ['service_b81f8665be684648879a70001668b78b']

In [33]:
df1.loc['service_b97a3762d9124e8f93a32296c23f1d62','name'] + df1.loc['service_b97a3762d9124e8f93a32296c23f1d62','address'][:10]

"Joe's Crab ShackGround Flo"

In [34]:
#inputlist = ['The Cheesecake Factory','Catch 22','Joe\'s Crab Shack']
df1 = populateJSDWithID(inputlist,df1)
df1.head()

,topicDistribution,name,address,Taqado Mexican Kitchen East Food Court
id,,,,
service_00026cb6595b4e93bb530c64bb4df92f,"[0.0005571014085451798, 0.003470435993264386, ...",Lywa Oriental Foods,"Food Court, Al Barsha Mall, Barsha 2, Al Barsh...",0.330351
service_000726bc5df5449aa1b4555d54b754ff,"[0.00029594279024264676, 0.004474207121988655,...",Sprouts Restaurant & Cafe,"Shop 2, Eros House Badri Building, Near Abidos...",0.309575
service_000b662279ce4ade9e3fd48f912c7aee,"[8.036530215281802e-05, 0.01300210717370562, 8...",Costa Coffee,"Premier Inn, Dubai Investment Park, Dubai",0.351959
service_00199d7edaa14e9285d450f516538ba5,"[1.1849668326396843e-05, 4.465561080808407e-05...",Hayz,"Ground Floor, JLT Cluster R, Jumeirah Lake Tow...",0.307072
service_001bd5d853a8463b987ac5942a70436a,"[0.0003507513219250869, 0.00249677231122796, 3...",Qasr Al Jabal,"EPPCO Station, Damascus Street, Al Qusais Indu...",0.385435


In [35]:
df1.columns

Index(['topicDistribution', 'name', 'address',
       'Taqado Mexican Kitchen East Food Court'],
      dtype='object')

In [36]:
df1[['name', 'address', 'Taqado Mexican Kitchen East Food Court']].to_csv("/home/krowd/Desktop/taqado_jsd2.csv",header=True)

In [13]:
df1[['name',
 'address',
 'Prime68 - JW Marriott Marquis Level 68, JW Ma',
 'Frevo - Fairmont The Palm Fairmont The Pa',
 'Couqley French Bistro & Bar Laguna Tower, M']].to_excel("JSD-Prime68-Frevo-Couqley.xlsx",header=True,index=False)

In [14]:
df1.shape

(8334, 6)

In [12]:
df1.columns.tolist()

['topicDistribution',
 'name',
 'address',
 'Prime68 - JW Marriott Marquis Level 68, JW Ma',
 'Frevo - Fairmont The Palm Fairmont The Pa',
 'Couqley French Bistro & Bar Laguna Tower, M']

In [41]:
#df1.reset_index('id',drop=False,inplace=True)
df1[['name', 'address',
 "Joe's Crab Shack Ground Floor, F",
 "Joe's Crab Shack Level 1, Behind",
 'Catch 22 Shop 07, South,',
 'Catch 22 The Beach, Jume',
 'The Cheesecake Factory The Beach, Jume',
 'The Cheesecake Factory Ground Level, N',
 'The Cheesecake Factory Ground Level, B',
 'The Cheesecake Factory Level 1, Dubai ']].to_excel("JSD-cheesecake-catch22-joe-.xlsx",header=True,index=False)

In [158]:
df1.columns = ['topicDistribution',
 'name',
 'address',
 'Bok Bok-1',
 'Desi Baithak Restaurant-1',
 'Hot Bites Cafeteria-1']
df1.reset_index(drop=False,inplace=True)
df1.head()

,id,topicDistribution,name,address,Bok Bok-1,Desi Baithak Restaurant-1,Hot Bites Cafeteria-1
0,service_00145e9860e08c213bdfde278d9f0c6d3a8b,"[1.2351185798635113e-05, 1.9724878588438384e-0...",Kiroran Hot Pot,"Ground Floor, Block A, Centurion Star Tower, P...",0.452496,0.493870,0.456605
1,service_002e9a4735cadbf879b9d2bf4961bc761a65,"[0.0002247507479322277, 0.00035892757893042187...",Desi Baithak Restaurant,"Building P1, France Cluster, International Cit...",0.478968,0.000000,0.289532
2,service_0034f4f62c430ba0a37cf499dfc194d7defc,"[0.00013346565567594627, 0.0002131450288947382...",Sky Lounge - Two Seasons Hotel & Apartments,"Two Seasons Hotel & Apartments, Dubai Media Ci...",0.521880,0.377106,0.416903
3,service_00363941907558b86855872577449f684a17,"[0.00025841676297077155, 0.0004126922999878284...",Hot Bites Cafeteria,"Building V 08, Russia Cluster, International C...",0.349494,0.289532,0.000000
4,service_003f05177b774c1b85c704f538996d814efa,"[2.253860147595431e-06, 3.5994210184703477e-06...",Vasanta Bhavan Vegetarian Restaurant,"Ground Floor, Twin Tower Building, Al Nahda, D...",0.494824,0.336626,0.380883


In [159]:
df1 = df1[['id',
 'name',
 'Bok Bok-1',
 'Desi Baithak Restaurant-1',
 'Hot Bites Cafeteria-1']]
df1.head()

,id,name,Bok Bok-1,Desi Baithak Restaurant-1,Hot Bites Cafeteria-1
0,service_00145e9860e08c213bdfde278d9f0c6d3a8b,Kiroran Hot Pot,0.452496,0.493870,0.456605
1,service_002e9a4735cadbf879b9d2bf4961bc761a65,Desi Baithak Restaurant,0.478968,0.000000,0.289532
2,service_0034f4f62c430ba0a37cf499dfc194d7defc,Sky Lounge - Two Seasons Hotel & Apartments,0.521880,0.377106,0.416903
3,service_00363941907558b86855872577449f684a17,Hot Bites Cafeteria,0.349494,0.289532,0.000000
4,service_003f05177b774c1b85c704f538996d814efa,Vasanta Bhavan Vegetarian Restaurant,0.494824,0.336626,0.380883


# Only Zomato Dubai with 100 topics

In [138]:
import ast
with open('zomato-dubai-reviews-100-topic-composition-spark.json') as f:
    data = ast.literal_eval(f.read())
df2 = pd.DataFrame(data)
print(df2.shape)
df2.head()

(5872, 2)


,id,topicDistribution
0,service_00145e9860e08c213bdfde278d9f0c6d3a8b,"[2.242663059343371e-06, 4.120067487135681e-05,..."
1,service_002e9a4735cadbf879b9d2bf4961bc761a65,"[4.3227136148640266e-05, 0.2084636093577844, 3..."
2,service_0034f4f62c430ba0a37cf499dfc194d7defc,"[2.503246243345869e-05, 0.0004598793125224771,..."
3,service_00363941907558b86855872577449f684a17,"[5.0173467788402634e-05, 0.0009217527015064493..."
4,service_003f05177b774c1b85c704f538996d814efa,"[4.081586007762381e-07, 7.498411202016848e-06,..."


In [139]:
df2 = df2.merge(rest_df,on='id',how='inner')
print("Shape ",df2.shape)
df2.set_index('id',inplace=True)
df2.head()

Shape  (5872, 4)


,topicDistribution,name,address
id,,,
service_00145e9860e08c213bdfde278d9f0c6d3a8b,"[2.242663059343371e-06, 4.120067487135681e-05,...",Kiroran Hot Pot,"Ground Floor, Block A, Centurion Star Tower, P..."
service_002e9a4735cadbf879b9d2bf4961bc761a65,"[4.3227136148640266e-05, 0.2084636093577844, 3...",Desi Baithak Restaurant,"Building P1, France Cluster, International Cit..."
service_0034f4f62c430ba0a37cf499dfc194d7defc,"[2.503246243345869e-05, 0.0004598793125224771,...",Sky Lounge - Two Seasons Hotel & Apartments,"Two Seasons Hotel & Apartments, Dubai Media Ci..."
service_00363941907558b86855872577449f684a17,"[5.0173467788402634e-05, 0.0009217527015064493...",Hot Bites Cafeteria,"Building V 08, Russia Cluster, International C..."
service_003f05177b774c1b85c704f538996d814efa,"[4.081586007762381e-07, 7.498411202016848e-06,...",Vasanta Bhavan Vegetarian Restaurant,"Ground Floor, Twin Tower Building, Al Nahda, D..."


In [140]:
inputlist = ['Bok Bok','Desi Baithak Restaurant','Hot Bites Cafeteria']
df2 = populateJSD(inputlist,df2)
df2.head()

,topicDistribution,name,address,Bok Bok,Desi Baithak Restaurant,Hot Bites Cafeteria
id,,,,,,
service_00145e9860e08c213bdfde278d9f0c6d3a8b,"[2.242663059343371e-06, 4.120067487135681e-05,...",Kiroran Hot Pot,"Ground Floor, Block A, Centurion Star Tower, P...",0.471600,0.489710,0.453298
service_002e9a4735cadbf879b9d2bf4961bc761a65,"[4.3227136148640266e-05, 0.2084636093577844, 3...",Desi Baithak Restaurant,"Building P1, France Cluster, International Cit...",0.421779,0.000000,0.420909
service_0034f4f62c430ba0a37cf499dfc194d7defc,"[2.503246243345869e-05, 0.0004598793125224771,...",Sky Lounge - Two Seasons Hotel & Apartments,"Two Seasons Hotel & Apartments, Dubai Media Ci...",0.554937,0.396357,0.380889
service_00363941907558b86855872577449f684a17,"[5.0173467788402634e-05, 0.0009217527015064493...",Hot Bites Cafeteria,"Building V 08, Russia Cluster, International C...",0.501593,0.420909,0.000000
service_003f05177b774c1b85c704f538996d814efa,"[4.081586007762381e-07, 7.498411202016848e-06,...",Vasanta Bhavan Vegetarian Restaurant,"Ground Floor, Twin Tower Building, Al Nahda, D...",0.496488,0.451522,0.402638


In [160]:
df2.columns = ['topicDistribution',
 'name',
 'address',
 'Bok Bok-2',
 'Desi Baithak Restaurant-2',
 'Hot Bites Cafeteria-2']
df2.reset_index(drop=False,inplace=True)
df2.head()

,id,topicDistribution,name,address,Bok Bok-2,Desi Baithak Restaurant-2,Hot Bites Cafeteria-2
0,service_00145e9860e08c213bdfde278d9f0c6d3a8b,"[2.242663059343371e-06, 4.120067487135681e-05,...",Kiroran Hot Pot,"Ground Floor, Block A, Centurion Star Tower, P...",0.471600,0.489710,0.453298
1,service_002e9a4735cadbf879b9d2bf4961bc761a65,"[4.3227136148640266e-05, 0.2084636093577844, 3...",Desi Baithak Restaurant,"Building P1, France Cluster, International Cit...",0.421779,0.000000,0.420909
2,service_0034f4f62c430ba0a37cf499dfc194d7defc,"[2.503246243345869e-05, 0.0004598793125224771,...",Sky Lounge - Two Seasons Hotel & Apartments,"Two Seasons Hotel & Apartments, Dubai Media Ci...",0.554937,0.396357,0.380889
3,service_00363941907558b86855872577449f684a17,"[5.0173467788402634e-05, 0.0009217527015064493...",Hot Bites Cafeteria,"Building V 08, Russia Cluster, International C...",0.501593,0.420909,0.000000
4,service_003f05177b774c1b85c704f538996d814efa,"[4.081586007762381e-07, 7.498411202016848e-06,...",Vasanta Bhavan Vegetarian Restaurant,"Ground Floor, Twin Tower Building, Al Nahda, D...",0.496488,0.451522,0.402638


In [162]:
df2 = df2[['id',
 'name',
 'Bok Bok-2',
 'Desi Baithak Restaurant-2',
 'Hot Bites Cafeteria-2']]
df2.head()

,id,name,Bok Bok-2,Desi Baithak Restaurant-2,Hot Bites Cafeteria-2
0,service_00145e9860e08c213bdfde278d9f0c6d3a8b,Kiroran Hot Pot,0.471600,0.489710,0.453298
1,service_002e9a4735cadbf879b9d2bf4961bc761a65,Desi Baithak Restaurant,0.421779,0.000000,0.420909
2,service_0034f4f62c430ba0a37cf499dfc194d7defc,Sky Lounge - Two Seasons Hotel & Apartments,0.554937,0.396357,0.380889
3,service_00363941907558b86855872577449f684a17,Hot Bites Cafeteria,0.501593,0.420909,0.000000
4,service_003f05177b774c1b85c704f538996d814efa,Vasanta Bhavan Vegetarian Restaurant,0.496488,0.451522,0.402638


# Zomato Dubai with Trip London and 180 topics

In [181]:
import ast
with open('zomato-dubai-trip_london-reviews-topic-composition-spark.json') as f:
    data = ast.literal_eval(f.read())
df3 = pd.DataFrame(data)
print(df3.shape)
df3.head()

(23494, 2)


,id,topicDistribution
0,service_002e9a4735cadbf879b9d2bf4961bc761a65,"[0.044090708666452885, 0.0013756603136030098, ..."
1,service_0034f4f62c430ba0a37cf499dfc194d7defc,"[0.1038242919477415, 0.0008143128432563785, 0...."
2,service_00363941907558b86855872577449f684a17,"[0.012967111064250877, 0.001583594097263813, 0..."
3,service_003f05177b774c1b85c704f538996d814efa,"[0.002725217672174117, 1.3688660162002992e-05,..."
4,service_0042dafae20210076592731ce0d80dcf8672,"[0.0010415548796112637, 0.00012719873772624356..."


In [182]:
df3.set_index('id',inplace=True)

In [183]:
df3.head()

,topicDistribution
id,
service_002e9a4735cadbf879b9d2bf4961bc761a65,"[0.044090708666452885, 0.0013756603136030098, ..."
service_0034f4f62c430ba0a37cf499dfc194d7defc,"[0.1038242919477415, 0.0008143128432563785, 0...."
service_00363941907558b86855872577449f684a17,"[0.012967111064250877, 0.001583594097263813, 0..."
service_003f05177b774c1b85c704f538996d814efa,"[0.002725217672174117, 1.3688660162002992e-05,..."
service_0042dafae20210076592731ce0d80dcf8672,"[0.0010415548796112637, 0.00012719873772624356..."


In [184]:
df3.shape

(23494, 1)

In [186]:
df3 = df3[df3.index.isin(df2.id.to_list())]

In [187]:
df3.shape

(5871, 1)

In [188]:
df3 = df3.merge(rest_df,on='id',how='inner')
print("Shape ",df3.shape)
df3.set_index('id',inplace=True)
df3.head()

Shape  (5871, 4)


,topicDistribution,name,address
id,,,
service_002e9a4735cadbf879b9d2bf4961bc761a65,"[0.044090708666452885, 0.0013756603136030098, ...",Desi Baithak Restaurant,"Building P1, France Cluster, International Cit..."
service_0034f4f62c430ba0a37cf499dfc194d7defc,"[0.1038242919477415, 0.0008143128432563785, 0....",Sky Lounge - Two Seasons Hotel & Apartments,"Two Seasons Hotel & Apartments, Dubai Media Ci..."
service_00363941907558b86855872577449f684a17,"[0.012967111064250877, 0.001583594097263813, 0...",Hot Bites Cafeteria,"Building V 08, Russia Cluster, International C..."
service_003f05177b774c1b85c704f538996d814efa,"[0.002725217672174117, 1.3688660162002992e-05,...",Vasanta Bhavan Vegetarian Restaurant,"Ground Floor, Twin Tower Building, Al Nahda, D..."
service_0042dafae20210076592731ce0d80dcf8672,"[0.0010415548796112637, 0.00012719873772624356...",Bok Bok,Delivery Only


In [189]:
df3.shape

(5871, 3)

In [190]:
inputlist = ['Bok Bok','Desi Baithak Restaurant','Hot Bites Cafeteria']
df3 = populateJSD(inputlist,df3)
df3.head()

,topicDistribution,name,address,Bok Bok,Desi Baithak Restaurant,Hot Bites Cafeteria
id,,,,,,
service_002e9a4735cadbf879b9d2bf4961bc761a65,"[0.044090708666452885, 0.0013756603136030098, ...",Desi Baithak Restaurant,"Building P1, France Cluster, International Cit...",0.318830,0.000000,0.303005
service_0034f4f62c430ba0a37cf499dfc194d7defc,"[0.1038242919477415, 0.0008143128432563785, 0....",Sky Lounge - Two Seasons Hotel & Apartments,"Two Seasons Hotel & Apartments, Dubai Media Ci...",0.509289,0.413276,0.407264
service_00363941907558b86855872577449f684a17,"[0.012967111064250877, 0.001583594097263813, 0...",Hot Bites Cafeteria,"Building V 08, Russia Cluster, International C...",0.299654,0.303005,0.000000
service_003f05177b774c1b85c704f538996d814efa,"[0.002725217672174117, 1.3688660162002992e-05,...",Vasanta Bhavan Vegetarian Restaurant,"Ground Floor, Twin Tower Building, Al Nahda, D...",0.344601,0.386226,0.341108
service_0042dafae20210076592731ce0d80dcf8672,"[0.0010415548796112637, 0.00012719873772624356...",Bok Bok,Delivery Only,0.000000,0.318830,0.299654


In [191]:
df3.columns = ['topicDistribution',
 'name',
 'address',
 'Bok Bok-3',
 'Desi Baithak Restaurant-3',
 'Hot Bites Cafeteria-3']
df3.reset_index(drop=False,inplace=True)
df3.head()

,id,topicDistribution,name,address,Bok Bok-3,Desi Baithak Restaurant-3,Hot Bites Cafeteria-3
0,service_002e9a4735cadbf879b9d2bf4961bc761a65,"[0.044090708666452885, 0.0013756603136030098, ...",Desi Baithak Restaurant,"Building P1, France Cluster, International Cit...",0.318830,0.000000,0.303005
1,service_0034f4f62c430ba0a37cf499dfc194d7defc,"[0.1038242919477415, 0.0008143128432563785, 0....",Sky Lounge - Two Seasons Hotel & Apartments,"Two Seasons Hotel & Apartments, Dubai Media Ci...",0.509289,0.413276,0.407264
2,service_00363941907558b86855872577449f684a17,"[0.012967111064250877, 0.001583594097263813, 0...",Hot Bites Cafeteria,"Building V 08, Russia Cluster, International C...",0.299654,0.303005,0.000000
3,service_003f05177b774c1b85c704f538996d814efa,"[0.002725217672174117, 1.3688660162002992e-05,...",Vasanta Bhavan Vegetarian Restaurant,"Ground Floor, Twin Tower Building, Al Nahda, D...",0.344601,0.386226,0.341108
4,service_0042dafae20210076592731ce0d80dcf8672,"[0.0010415548796112637, 0.00012719873772624356...",Bok Bok,Delivery Only,0.000000,0.318830,0.299654


In [192]:
df3 = df3[['id',
 'name',
 'Bok Bok-3',
 'Desi Baithak Restaurant-3',
 'Hot Bites Cafeteria-3']]
df3.head()

,id,name,Bok Bok-3,Desi Baithak Restaurant-3,Hot Bites Cafeteria-3
0,service_002e9a4735cadbf879b9d2bf4961bc761a65,Desi Baithak Restaurant,0.318830,0.000000,0.303005
1,service_0034f4f62c430ba0a37cf499dfc194d7defc,Sky Lounge - Two Seasons Hotel & Apartments,0.509289,0.413276,0.407264
2,service_00363941907558b86855872577449f684a17,Hot Bites Cafeteria,0.299654,0.303005,0.000000
3,service_003f05177b774c1b85c704f538996d814efa,Vasanta Bhavan Vegetarian Restaurant,0.344601,0.386226,0.341108
4,service_0042dafae20210076592731ce0d80dcf8672,Bok Bok,0.000000,0.318830,0.299654


In [196]:
final_df = pd.merge(pd.merge(df1,df2,on=['id','name']),df3,on=['id','name'])

In [197]:
final_df.head()

,id,name,Bok Bok-1,Desi Baithak Restaurant-1,Hot Bites Cafeteria-1,Bok Bok-2,Desi Baithak Restaurant-2,Hot Bites Cafeteria-2,Bok Bok-3,Desi Baithak Restaurant-3,Hot Bites Cafeteria-3
0,service_002e9a4735cadbf879b9d2bf4961bc761a65,Desi Baithak Restaurant,0.478968,0.000000,0.289532,0.421779,0.000000,0.420909,0.318830,0.000000,0.303005
1,service_0034f4f62c430ba0a37cf499dfc194d7defc,Sky Lounge - Two Seasons Hotel & Apartments,0.521880,0.377106,0.416903,0.554937,0.396357,0.380889,0.509289,0.413276,0.407264
2,service_00363941907558b86855872577449f684a17,Hot Bites Cafeteria,0.349494,0.289532,0.000000,0.501593,0.420909,0.000000,0.299654,0.303005,0.000000
3,service_003f05177b774c1b85c704f538996d814efa,Vasanta Bhavan Vegetarian Restaurant,0.494824,0.336626,0.380883,0.496488,0.451522,0.402638,0.344601,0.386226,0.341108
4,service_0042dafae20210076592731ce0d80dcf8672,Bok Bok,0.000000,0.478968,0.349494,0.000000,0.421779,0.501593,0.000000,0.318830,0.299654


In [198]:
final_df.to_excel("jsdComaprisionFor3situations.xlsx",index=False,header=True)